# Question 1 Modelling fishies
This notebook is a toy to mess around with the solutions of the simple fish population model

Import required libraries

In [ ]:
import numpy as np
from scipy.integrate import solve_ivp  # Used to numerically solve the ODE
import matplotlib.pyplot as plt

In [ ]:
# Library to allow for interactive plots
from IPython.display import display
from ipywidgets import interact, widgets, interactive

Define the function (right hand side of the eqution)

In [ ]:
def fish_model(t=0, P=1, a=1, N=1, H=1):
    return a * P * (1 - P/N) - H

And now the simplified change of variables (CV) version

In [ ]:
def fish_model_cv(t=0, x=1, h=1):
    return x * (1 - x) - h

We define the equilibria and the stability properties for the simplified version

In [ ]:
# zeros of above function
def calculate_equlibria(h=1):
    eq1 = (1 + np.sqrt(1 - 4 * h)) / 2
    eq2 = (1 - np.sqrt(1 - 4 * h)) / 2
    return (eq1, eq2)

# Stability of solutions (value of the derivative)
def calculate_stability(eq):
    eq1, eq2 = eq
    output = (
        1 - 2 * eq1,
        1 - 2 * eq2,
    )
    return output

Lets plot this for different values of $a,~b,~r$

In [ ]:
%matplotlib notebook

# Plotting simplified equation

In [ ]:
x = np.linspace(-0.2, 2, 100)

fig, ax = plt.subplots(figsize=(8, 5))
line, = ax.plot(x, fish_model_cv(x=x))  # Initial plot
ax.set_xlabel('x')
ax.set_ylabel('dx/dt')

# set zero line
ax.hlines(0, xmin=-0.2, xmax=2, color='lightgrey')
ax.vlines(0, ymin=-0.2, ymax=2, color='lightgrey')
ax.set_xlim(-0.2, 1.2)
ax.set_ylim(-0.2, 1.2)


def update1(h):
    line.set_ydata(fish_model_cv(x=x, h=h))
    fig.canvas.draw_idle()
    
slid1 = interactive(update1, 
         h=widgets.FloatSlider(min=-5, max=5, step=0.05, value=0.25))
display(slid1)

## Plotting the phase space
This plots a bunch of trajectories starting at $t=0$, with $x(t=0)\in[-0.2, 2.2]$, and shows how these initial conditions evolve in time. Note the colours of the trajectories have no meaning and are just to help keep them seperate. The equilibria are shown by grey lines, where the unstable solution is dotted.

In [ ]:
fig, ax = plt.subplots(figsize=(8, 5))

t = np.linspace(0, 2, 50)

values  = np.linspace(0.1, 2.2, 10)                      
vcolors = plt.cm.autumn_r(np.linspace(0.3, 1., len(values)))
                          
# Plot trajectories
lines = list()
for v, col in zip(values, vcolors):
    X = solve_ivp(fish_model_cv, (0, 2), [v], args=(-1,), t_eval=t)
    line1, = ax.plot(X.t, X.y.T, color=col)
    lines.append(line1)
    
# plot Equilibria
lines_eq = list()
eq = calculate_equlibria()
for q in eq:
    line1, = ax.plot(t, q * np.ones(len(t)), color='lightgrey')
    lines_eq.append(line1)

def update3(h):
    print("1^2-4h: " + str(1 ** 2 - 4 * h))
    for i, v in enumerate(values):
        X = solve_ivp(fish_model_cv, (0, 2), [v], args=(h,), t_eval=t)
        lines[i].set_ydata(np.maximum(X.y.T, -1))
    
    eq = calculate_equlibria(h)
    stab = calculate_stability(eq)
    
    for i, (q, s) in enumerate(zip(eq, stab)):
        if s > 0:
            line_sty = ":"
        else:
            line_sty = "solid"
        lines_eq[i].set_ydata(q * np.ones(len(t)))
        lines_eq[i].set_linestyle(line_sty)
    
    fig.canvas.draw_idle()


plt.xlabel('t')
plt.ylabel('x')

slid3 = interactive(update3, 
         h=widgets.FloatSlider(min=-1.5, max=0.4, step=0.05, value=-1))
display(slid3)

# Improved Fish Model

In [ ]:
def fish_model_imporved(t=0, x=1, h=1, b=1):
    return x * (1 - x) - h * x / (b + x)

In [ ]:
def fish_model_plotting(t=0, x=1, h=1, b=1):
    sol = fish_model_imporved(t, x, h, b)
    
    # Code to remove asymptots for plotting purposes
    ab_sol = np.abs(sol)
    if np.sum(ab_sol > 5) > 0:
        sol[ab_sol > 5] = np.nan
    return sol

In [ ]:
# zeros of above function
def calculate_equlibria(h=1, b=1):
    eq1 = 0
    eq2 = (-(b - 1) + np.sqrt((b - 1)**2 - 4 * (h - b))) / 2
    eq3 = (-(b - 1) - np.sqrt((b - 1)**2 - 4 * (h - b))) / 2
    return (eq1, eq2, eq3)

# Stability of solutions (value of the derivative)
def calculate_stability(eqs, h=1, b=1):
    output = [1 - 2 * eq - h * b / (b + eq)**2 for eq in eqs]
    return output

In [ ]:
x = np.linspace(-0.2, 3, 200)

fig, ax = plt.subplots(figsize=(8, 5))

line, = ax.plot(x, fish_model_plotting(x=x))  # Initial plot
ax.set_xlabel('x')
ax.set_ylabel('dx/dt')

# set zero line
ax.hlines(0, xmin=-0.2, xmax=3, color='lightgrey')
ax.vlines(0, ymin=-0.2, ymax=2, color='lightgrey')
ax.set_xlim(-0.2, 3.2)
ax.set_ylim(-0.2, 1.2)


def update1(h, b):
    print("(b-1)^2: " + str((b - 1)**2))
    print("4(h - b): " + str(4 * (h - b)))
    line.set_ydata(fish_model_plotting(x=x, h=h, b=b))
    fig.canvas.draw_idle()
    
slid1 = interactive(update1, 
         h=widgets.FloatSlider(min=-5, max=5, step=0.01, value=-1),
         b=widgets.FloatSlider(min=-5, max=5, step=0.01, value=-1))
display(slid1)

In [ ]:
fig, ax = plt.subplots(figsize=(8, 5))

t_max = 1
t = np.linspace(0, t_max, 50)

values  = np.linspace(-0.4, 2.2, 10)                      
vcolors = plt.cm.autumn_r(np.linspace(0.3, 1., len(values)))
                          
# Plot trajectories
lines = list()
for v, col in zip(values, vcolors):
    X = solve_ivp(fish_model_imporved, (0, t_max), [v], args=(1,1), t_eval=t)
    line1, = ax.plot(X.t, X.y.T, color=col)
    lines.append(line1)
    
# plot Equilibria
lines_eq = list()
eq = calculate_equlibria()
for q in eq:
    line1, = ax.plot(t, q * np.ones(len(t)), color='lightgrey')
    lines_eq.append(line1)

def update3(h, b):
    print("1^2-4h: " + str(1 ** 2 - 4 * h))
    for i, v in enumerate(values):
        X = solve_ivp(fish_model_imporved, (0, t_max), [v], args=(h,b), t_eval=t, method='RK45')
        if X.success:
            lines[i].set_ydata(X.y.T)
    
    eq = calculate_equlibria(h, b)
    stab = calculate_stability(eq, h, b)
    print(stab)
    
    for i, (q, s) in enumerate(zip(eq, stab)):
        if s > 0:
            line_sty = ":"
        else:
            line_sty = "solid"
        lines_eq[i].set_ydata(q * np.ones(len(t)))
        lines_eq[i].set_linestyle(line_sty)
    
    fig.canvas.draw_idle()


plt.xlabel('t')
plt.ylabel('x')

slid3 = interactive(update3, 
        h=widgets.FloatSlider(min=-2, max=4, step=0.25, value=1),
        b=widgets.FloatSlider(min=-2, max=4, step=0.25, value=0.5))
display(slid3)